In [2]:
import numpy as np
import pandas as pd
import json
import halfspace.projections as hsp
from halfspace.scripts import *

- load point dataframes
- load geojson tris
- get point stresses
- average
- make new df of average stresses, using tri center coords
- merge w/ other fault df(s)

In [3]:
tib_dog_pts = pd.read_csv('../data/fault_data/tib_dog_pts_df.csv')
gurla_pts = pd.read_csv('../data/fault_data/gurla_pts_df.csv')

In [12]:
tdg_pts = pd.concat((tib_dog_pts, gurla_pts), axis=0, ignore_index=True)

tdg_pts.columns = ['tri', 'vertex', 'east', 'north', 'depth', 'fault_name']

In [13]:
tdg_pts.tail()

,tri,vertex,east,north,depth,fault_name
2931,340,m,-54140.576926,3406463.683422,-18788.333333,gurla
2932,341,a,-57886.265887,3407087.521804,-20455.000000,gurla
2933,341,b,-54899.206517,3407362.546041,-20455.000000,gurla
2934,341,c,-55140.956705,3405082.406588,-15455.000000,gurla
2935,341,m,-55975.476370,3406510.824811,-18788.333333,gurla


In [8]:
with open('../data/fault_data/tib_dog_tris.geojson') as f:
    tdt = json.load(f)
    
with open('../data/fault_data/gurla_tris.geojson') as f:
    gt = json.load(f)

In [17]:
stress_file = '../../../nepal_2015/stress_arrays/nepal_topo_stress.h5'
stress_meta_file = '../../../nepal_2015/stress_arrays/stress_calcs_meta.json'
dem_meta_file = '../../../nepal_2015/data/dem/s_tibet_dem_meta.json'

In [18]:
tdg = calc_topo_stresses_at_pts(tdg_pts, stress_file=stress_file,
                                dem_meta_file=dem_meta_file,
                                stress_meta_file=stress_meta_file,
                                resolve_stresses=False)

interpolating zz stresses
interpolating xy stresses
interpolating xz stresses
interpolating yz stresses
interpolating xx stresses
interpolating yy stresses


In [19]:
tdg.head()

,tri,vertex,east,north,depth,fault_name,zz_stress,xy_stress,xz_stress,yz_stress,xx_stress,yy_stress
0,0,a,124496.100000,3172901.700000,4000.000000,tib_dog,0,0,0,0,0,0
1,0,b,125070.300000,3172690.800000,3743.000000,tib_dog,0,0,0,0,0,0
2,0,c,124256.800000,3172817.900000,4156.000000,tib_dog,0,0,0,0,0,0
3,0,m,124607.733333,3172803.466667,3966.333333,tib_dog,0,0,0,0,0,0
4,1,a,124058.100000,3173298.000000,4000.000000,tib_dog,0,0,0,0,0,0


In [42]:
tdg_avg = tdg.groupby(('fault_name', 'tri')).mean()[['zz_stress',
                                                     'xy_stress',
                                           'xz_stress','yz_stress',
                                           'xx_stress','yy_stress']]

In [43]:
tdg_avg.head()

zz_stress  xy_stress  xz_stress  yz_stress  xx_stress  \
fault_name tri                                                           
gurla      0    122.365046   1.262838  -3.130255  -7.833799  67.215418   
           1    122.802498   1.562838  -2.546067  -7.169587  68.833418   
           2    123.452559   2.246759  -1.578494  -5.545806  72.404882   
           3    123.926082   2.731499  -1.128569  -4.596917  74.340048   
           4    122.374934   1.156617  -3.483587  -7.801336  66.988821   

                yy_stress  
fault_name tri             
gurla      0    67.963298  
           1    69.845652  
           2    74.020897  
           3    76.356889  
           4    67.718253

In [44]:
tdg_avg['i'] = range(tdg_avg.shape[0])
tdg_avg['fault_name'],tdg_avg['tri'] = zip(*tdg_avg.index.values)

In [45]:
tdg_avg.set_index('i', inplace=True)

In [49]:
tdg_avg.head(), tdg_avg.tail()

(    zz_stress  xy_stress  xz_stress  yz_stress  xx_stress  yy_stress  \
 i                                                                      
 0  122.365046   1.262838  -3.130255  -7.833799  67.215418  67.963298   
 1  122.802498   1.562838  -2.546067  -7.169587  68.833418  69.845652   
 2  123.452559   2.246759  -1.578494  -5.545806  72.404882  74.020897   
 3  123.926082   2.731499  -1.128569  -4.596917  74.340048  76.356889   
 4  122.374934   1.156617  -3.483587  -7.801336  66.988821  67.718253   
 
   fault_name  tri  
 i                  
 0      gurla    0  
 1      gurla    1  
 2      gurla    2  
 3      gurla    3  
 4      gurla    4  ,
       zz_stress  xy_stress  xz_stress  yz_stress  xx_stress  yy_stress  \
 i                                                                        
 729  102.925521  -0.030489  -7.277902 -10.392833  72.365598  74.368759   
 730  101.775826  -0.468165  -2.945500 -10.891380  73.363035  77.416506   
 731  100.918067  -0.009567  -4.248177 

In [50]:
fd = {'gurla': gt, 'tib_dog':tdt} # fault dict

In [51]:
gt['features'][0]

{'geometry': {'coordinates': [[[-55783.583006836, 3342716.71580536, -14985.0],
    [-54677.8965490291, 3339938.7432105276, -19768.0],
    [-56725.8969340146, 3341331.705430983, -19747.0],
    [-55783.583006836, 3342716.71580536, -14985.0]]],
  'type': 'Polygon'},
 'properties': {'POINTA': 130.0,
  'POINTB': 204.0,
  'POINTC': 203.0,
  'area_sq_km': 6.251594281640157,
  'center': [-55729.12549662657, 3341329.0548156234, -18166.666666666668],
  'dip': 70.61925626388545,
  'rake': 73.88142642292091,
  'rake_err': 10,
  'strike': 325.9490289153339},
 'type': 'Feature'}

In [52]:
def get_row(row):
    g = fd[row.fault_name]
    p = g['features'][row.tri]['properties']
    o = {}
    o['east'], o['north'], o['depth'] = p['center']
    o['strike'] = p['strike']
    o['dip'] = p['dip']
    o['rake'] = p['rake']
    
    return pd.Series(o)

In [54]:
get_row(tdg_avg.iloc[0])

depth      -18166.666667
dip            70.619256
east       -55729.125497
north     3341329.054816
rake           73.881426
strike        325.949029
dtype: float64

In [61]:
tdg_avg.apply(get_row, axis=1).head()

,depth,dip,east,north,rake,strike
i,,,,,,
0,-18166.666667,70.619256,-55729.125497,3341329.054816,73.881426,325.949029
1,-16527.333333,73.934283,-54657.964479,3341155.562365,72.991457,319.135403
2,-13288.333333,68.460230,-53918.655475,3341974.752790,67.685005,318.813582
3,-11725.000000,69.749617,-54321.288308,3343089.493691,70.570696,321.458319
4,-18510.000000,70.715283,-57216.150551,3342311.810013,70.394943,319.837489


In [66]:
tri_stress = pd.merge(tdg_avg, tdg_avg.apply(get_row, axis=1),
                       left_index=True, right_index=True)

In [67]:
tri_stress.head()

,zz_stress,xy_stress,xz_stress,yz_stress,xx_stress,yy_stress,fault_name,tri,depth,dip,east,north,rake,strike
i,,,,,,,,,,,,,,
0,122.365046,1.262838,-3.130255,-7.833799,67.215418,67.963298,gurla,0,-18166.666667,70.619256,-55729.125497,3341329.054816,73.881426,325.949029
1,122.802498,1.562838,-2.546067,-7.169587,68.833418,69.845652,gurla,1,-16527.333333,73.934283,-54657.964479,3341155.562365,72.991457,319.135403
2,123.452559,2.246759,-1.578494,-5.545806,72.404882,74.020897,gurla,2,-13288.333333,68.460230,-53918.655475,3341974.752790,67.685005,318.813582
3,123.926082,2.731499,-1.128569,-4.596917,74.340048,76.356889,gurla,3,-11725.000000,69.749617,-54321.288308,3343089.493691,70.570696,321.458319
4,122.374934,1.156617,-3.483587,-7.801336,66.988821,67.718253,gurla,4,-18510.000000,70.715283,-57216.150551,3342311.810013,70.394943,319.837489


In [70]:
tri_stress.rename(columns={'tri':'point_index'}, inplace=True)

In [71]:
tri_stress = resolve_stresses(tri_stress)

In [73]:
tri_stress.shape

(734, 17)

In [74]:
tri_stress = tri_stress[tri_stress.tau_dd != 0.]

In [75]:
tri_stress.shape

(610, 17)

In [76]:
tri_stress.slip_m = 0.1